# **Import Libraries & Data Loading**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re # if u want to learn regex [https://regex101.com/]
import string
import random # Random number generators - Library for generating random numbers, selecting random elements, shuffling sequences, etc.
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_row", None)

In [2]:
!pip install sastrawi nlp-id

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 20.9 MB/s eta 0:00:00
  Created wheel for nlp-id: filename=nlp_id-0.1.15.0-py3-none-any.whl size=58153892 sha256=ab5bab9be03489067a0bac1beb87b7ee62017f0b0ca4feaaab64d84b0b89e64f
  Stored in directory: /root/.cache/pip/wheels/85/34/2a/deaf7b7896a1eed336b874e1a7732588d40bb7bdc0a824e73e
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=d1d75a62959730e25251191d9e1c503b14ee6ea6d303986b82fd011964c1807f
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built nlp-id wget
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully un

In [49]:
import nltk # Natural Language Toolkit - Library for natural language processing (NLP) tasks such as tokenization, stemming, tagging, parsing, and more.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm # A Fast, Extensible Progress Bar - Library for creating progress bars to monitor the progress of iterations or tasks.
from nltk.stem import WordNetLemmatizer, PorterStemmer

from nlp_id.lemmatizer import Lemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# Hapus @, hapus link, reply message nya di ilangin juga, non ascii ilangin atau replace. yang typo typo bisa maping manual, atau cari cara nya () spell checker namanya.
# Vectorize terus kemodel kalau udah slesai preprocessing (kalau mau manual)

# TF-IDF, Word2Vec, dll

# kalau bert itu mau kata kata yang utuh, jadi jangan terlalu aneh aneh untuk processing. jadi intinya processing sederhana

# Coba Finetune bert model, biar bisa sesuai dengan kasus yang sedang dihadapi

# terus ada lagi yang namanya sentence transformers, kalau bert itu kata kata masukin model, terus dapet labelnya apa. tpi kalau dsi sentence transformers itu ngambil vector di layer tertentu. misalkan ada kata kata, terus kitam auskin ke model, nah dia ngasih dalam bentuk vektor bukan labelnya, nah enaknya hasil vektor ini enaknya adalah, bisa digabungin oleh jumlah label lain, misalkan catboost dll nya.

# coba pakai indo bert tweet, bukan indobert biasa (nama model nya) atau coba dibandingkan aja. kalau mau pake bert tweet, jangan hapus hastag nya jangan diolah aneh aneh.

# Saran coba pakai catboost ajah. catboostclassifier(text_feature = ["kolommu"]) terus tinggal di fit aja.

# coba liat wordcloud untuk bantuan membedakan setiapa labelnya, jadi text yang paling gede(sering muncul) apakah ada perbedaan diantara label nya.

# bisa juga buat featrue extraction, tmabah kolom, is_debat, is_capres, nah jadi kayak 1 0 1 0 gitu.

# bisa juga pake post tagging, jadi nnti ada kolom baru, jumlah subject nya, jumlah adjectivenya dll lah.

# pertama coba baseline dlu, baru coba yang aneh aneh nya processing nya.

# ubah label yang inconsistent, coba coba aja. boleh kok processing data di label nya juga.

# catboost classifier for class imbalanced, scale_pos_weight. jadi menanggulangi class imbalance, coba coba aja sih.

In [5]:
def check_duplicates(dataframe):
    print("Duplicate Values (Top 10):")
    duplicate_values = dataframe[dataframe.duplicated()]
    print(f"Number of Duplicate Rows: {duplicate_values.shape[0]}")
    display(duplicate_values.head(10))

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

def basic_data_info(dataframe):
    print("Data Preview:")
    print("---------------------------")
    display(dataframe.head())

    print("\nGeneral Info:")
    print("---------------------------")
    print(dataframe.info())

    print("\nDescriptive Statistics:")
    print("---------------------------")
    display(dataframe.describe().T)

# **Load Data**

In [13]:
mainPath = "/content/drive/MyDrive/Colab Notebooks/2. Satria Data 2024/Pengerjaan Satria Data"
dataPath = os.path.join(mainPath, "Dataset-BDC-SatriaData-2024")
corpusPath = os.path.join(mainPath, "Corpus by TimTam")
cleanDataPath = os.path.join(mainPath, "Clean Dataset")

In [14]:
train = pd.read_csv(dataPath + "/dataset_penyisihan_bdc_2024.csv", sep=";")
test = pd.read_csv(dataPath + "/dataset_unlabeled_penyisihan_bdc_2024.csv", sep=";")
submissions = pd.read_csv(dataPath + "/template_jawaban_penyisihan_bdc_2024.csv", sep=";")
lexicon_corpus = pd.read_excel(corpusPath + "/colloquial-indonesian-lexicon.xlsx", sheet_name="Clean-Colloquial-Lexicon")
stopword_corpus = pd.read_excel(corpusPath + "/Stopword byTimTam.xlsx")

# **Simple Explore the Data**

In [ ]:
check_duplicates(train)
# drop duplicate entries considering all columns
train = train.drop_duplicates()

Duplicate Values (Top 10):
Number of Duplicate Rows: 381


,text,label
57,RT Abah Anies ingin mengangkat martabat petani...,Sosial Budaya
104,"RT Anak Muda Indonesia, the future of this nat...",Ideologi
145,"RT Pupuk bersubsidi langka, Tim Prabowo Gibran...",Ekonomi
146,RT Abah Anies ingin mengangkat martabat petani...,Sosial Budaya
189,RT Abah Anies ingin mengangkat martabat petani...,Sosial Budaya
220,RT Abah Anies ingin mengangkat martabat petani...,Sosial Budaya
234,"RT al fatihah buat Alm. Lambang Babar Purnomo,...",Pertahanan dan Keamanan
257,RT Kapitalisme neoliberal tak cocok untuk Indo...,Ideologi
298,"RT Anak Muda Indonesia, the future of this nat...",Ideologi
350,"RT According to Prabowo, Gaza is opressed beca...",Pertahanan dan Keamanan


In [ ]:
missing_data(train)

,Total,Percent
text,0,0.0
label,0,0.0


In [ ]:
basic_data_info(train)

Data Preview:
---------------------------


,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik



General Info:
---------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 4619 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4619 non-null   object
 1   label   4619 non-null   object
dtypes: object(2)
memory usage: 108.3+ KB
None

Descriptive Statistics:
---------------------------


,count,unique,top,freq
text,4619,4583,RT Abah Anies Janji Bakal Revisi UU KPK untuk ...,2
label,4619,8,Politik,2972


In [ ]:
train.label.value_counts()

label
Politik                    2972
Sosial Budaya               425
Ideologi                    343
Pertahanan dan Keamanan     331
Ekonomi                     310
Sumber Daya Alam            157
Demografi                    61
Geografi                     20
Name: count, dtype: int64

# **Feature Engineering**

> > ### ***Preprocessing Data***
---

### Wordcloud - For Every Label

In [ ]:
# Memisahkan teks berdasarkan label
texts_by_label = train.groupby('label')['text'].apply(lambda x: ' '.join(x)).to_dict()

# Custom tokenizer to split words
def simple_tokenizer(text):
    return text.split()

In [ ]:
# Vectorizer setup
vectorizer = CountVectorizer(tokenizer=simple_tokenizer, stop_words='english')

wordclouds = {}

# Membuat wordcloud untuk setiap label
for label, texts in texts_by_label.items():
    X = vectorizer.fit_transform([texts])
    word_freq = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))
    wordclouds[label] = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# Plotting the word clouds
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
axes = axes.flatten()

for ax, (label, wordcloud) in zip(axes, wordclouds.items()):
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(label, fontsize=20)
    ax.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# for i in train.sample(10, random_state=42)["text"]:
#   print(i)

### Text Processing **_[clean_text_1]_**

1. Mengubah semua karakter dalam teks menjadi **huruf kecil.**
2. Menghapus **'RT'**: Menggunakan pola \bRT\b untuk mencocokkan kata 'RT' sebagai kata tunggal.
3. Menghapus **link**: Menggunakan pola http\S+|www\S+ untuk menghapus link.
4. Menghapus pola **'[RE ...]'**: Menggunakan pola \[RE [^]]+\] untuk menghapus teks yang diawali dengan [RE dan diakhiri dengan ].
5. Menghapus pola **'@ ... ='**: Menggunakan pola @\S+= untuk menghapus teks yang diawali dengan @ dan diakhiri dengan =.
6. Menghapus **karakter non-ASCII (termasuk emoji)**: Menggunakan pola [^\x00-\x7F]+ untuk menghapus karakter non-ASCII.
7. Menghapus **spasi berlebih**: Menggunakan pola \s+ untuk mengganti spasi berlebih dengan satu spasi.
8. Menghapus **Tanda Baca**: Menggunakan translate string punctuation
9. Menghapus **Angka**: Menggunakan regex \b\d+\b untuk menghapus angka.
10. menghapus **single char**: Menggunakan regex "\b[a-zA-Z]\b" untuk menghapus karakter tunggal.
11. mengganti kata **slang** menjadi kata **formal**: mengganti kata slang dengan kata formal dalam teks menggunakan ekspresi reguler. Setiap pasangan kata slang dan formal dari slang_formal_pairs akan diaplikasikan ke teks.

In [ ]:
# for i in (lexicon_corpus[lexicon_corpus.isnull().any(axis=1)]).slang:
#   print(i)

In [ ]:
# drop nilai nan di dalam corpus lexicon, yang kata formalnya tidak ada
lexicon_corpus = lexicon_corpus.dropna(subset=['formal'])

def replace_slang(text, slang_formal_pairs):
    for slang, formal in slang_formal_pairs:
        text = re.sub(r'\b{}\b'.format(re.escape(slang)), formal, text)
    return text

slang_formal_pairs = list(zip(lexicon_corpus['slang'], lexicon_corpus['formal']))

In [ ]:
# Fungsi untuk membersihkan teks
def clean_text(text, slang_formal_pairs = slang_formal_pairs):
    # Lowercasing
    text = text.lower()
    # Menghapus 'RT'
    text = re.sub(r'\brt\b', '', text)
    # Menghapus link (link tweet)
    text = re.sub(r'http\S+|www\S+', '', text)
    # Menghapus pola '[RE ...]'
    text = re.sub(r'\[re [^]]+\]', '', text)
    # Menghapus pola '@ ... ='
    text = re.sub(r'@\S+=', '', text)
    # Menghapus karakter non-ASCII (termasuk emoji)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
# # Menghapus tanda baca dari kalimat
# text = text.translate(str.maketrans("", "", string.punctuation))
    # Menghapus tanda baca kecuali '#'
    text = re.sub(r'[{}]'.format(re.escape(string.punctuation.replace('#', ''))), '', text)
    # Menghapus angka
    text = re.sub(r'\b\d+\b', '', text)
    # Menghapus karakter tunggal (single character)
    text = re.sub(r"\b[a-zA-Z]\b", " ", text)
    # Mengganti kata-kata dengan kata yang sesuai dari lexicon_corpus
    text = replace_slang(text, slang_formal_pairs)

    return text

# Menerapkan fungsi ke kolom text
train['clean_text_1'] = train['text'].apply(clean_text)

In [ ]:
for tweet in train.sample(5, random_state= 2035)["text"]:
  print(tweet)

In [ ]:
for tweet in train.sample(5, random_state= 2035)["clean_text_1"]:
  print(tweet)

### Text Processing **_[clean_text_2]_**

1. Menggunakan **clean_text_1** ditambahkan dengan ***penghapusan untuk hastag nya***

In [ ]:
def remove_hashtag(text):
  text = re.sub(r'#\w+', '', text)
  # Menghapus spasi berlebih
  text = re.sub(r'\s+', ' ', text).strip()

  return text

In [ ]:
# Menerapkan fungsi ke kolom text
train['clean_text_2'] = train['clean_text_1'].apply(remove_hashtag)

### Text Processing **_[clean_text_3]_**

1. Menggunakan clean_text_2 ditambahkan dengan stop_word, lemmatization, Stemming (NLTK, nlp-id, Sastrawi)

In [ ]:
nltk.download(['stopwords', 'punkt', 'wordnet'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Initialize stopwords, lemmatizer, and stemmer

stop_words = set(stopwords.words('indonesian')) # NLTK
lemmatizer = Lemmatizer() # nlp-id

factory = StemmerFactory() # Sastrawi for stemming (Indonesian) nltk just have English Stemmer
stemmer_sastrawi = factory.create_stemmer()

# Add custom stopwords
custom_stopwords = {
    'yg', 'dgn', 'dg', 'd', 'kalo', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'tuh', 'nih',
    'sih', 'si', 'deh', 'loh', 'bakal', 'gak', 'ga', 'gitu', 'sampe', 'amp', 'rt',
    'dm', 'url', 'user', 'https', 'http', 'www', 'com', "..", "..."
}
stop_words.update(custom_stopwords)

In [ ]:
# Define the preprocessing function
def preprocess_text(text):

    # Tokenize the text into words
    words = text.split()

    # Remove stopwords and apply stemming and lemmatization
    processed_words = []
    for word in words:
        if word.lower() not in stop_words: # NLTK Stopword
            stemmed_word = stemmer_sastrawi.stem(word) # Sastrawi Stemming
            lemmatized_word = lemmatizer.lemmatize(stemmed_word) # npl-id
            processed_words.append(lemmatized_word)

    # Join the processed words back into a single string
    return " ".join(processed_words)

In [ ]:
# Apply the preprocessing function to the 'text' column
train['clean_text_3'] = train['clean_text_2'].apply(preprocess_text)

### Text Processing **_[clean_text_4]_**

1. Menggunakan **text awal**, namun untuk **processing tex**t nya akan dibantu oleh library **tweet-preprocessor** untuk processing text nya

In [ ]:
# !pip install tweet-preprocessor

In [ ]:
import preprocessor as p

In [ ]:
def preprocess_tweet(row):
    text = row['text']
    # Lowercasing
    text = text.lower()
    # Menghapus pola '@ ... ='
    text = re.sub(r'@\S+=', '', text)
    text = p.clean(text)
    return text

In [ ]:
train['clean_text_4'] = train.apply(preprocess_tweet, axis=1)

In [ ]:
train = train.reset_index(drop = True)

In [ ]:
# Define the labels
labels = [
    "Politik",
    "Sosial Budaya",
    "Ideologi",
    "Pertahanan dan Keamanan",
    "Ekonomi",
    "Sumber Daya Alam",
    "Demografi",
    "Geografi"
]

### Text Processing **_[clean_text_5]_**

Try to **handling** the **english and indonesian word**, here we should get the only **Indo word**, or **just get both** and **translate english into indonesian**

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=a52272428c811c11ccdd233ce9eb412dd7a7a6b25f004ead5b8bffbf29bf5fc5
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect, detect_langs

# Download stopwords
nltk.download('stopwords')
indonesian_stopwords = set(stopwords.words('indonesian'))
english_stopwords = set(stopwords.words('english'))

# Custom stopwords for Twitter
custom_stopwords = {
    'yg', 'dgn', 'dg', 'd', 'kalo', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'tuh', 'nih',
    'sih', 'si', 'deh', 'loh', 'bakal', 'gak', 'ga', 'gitu', 'sampe', 'amp', 'rt',
    'dm', 'url', 'user', 'https', 'http', 'www', 'com'
}

indonesian_stopwords.update(custom_stopwords)

# Define the function to preprocess and classify words
def preprocess_and_classify(text):
    # Remove mentions, hashtags, and URLs
    text = re.sub(r'@\w+|#\w+|http\S+', '', text)

    # Tokenize the text into words
    words = text.split()

    english_words = []
    other_words = []

    for word in words:
        # Remove stopwords
        if word.lower() in indonesian_stopwords or word.lower() in english_stopwords:
            continue

        # Classify the word
        try:
            lang = detect(word)
            if lang == 'en':
                english_words.append(word)
            else:
                other_words.append(word)
        except:
            other_words.append(word)

    return {
        'english': english_words,
        'other': other_words
    }

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Example sentence
sentence = train['clean_text_3'].iloc[4617]

# Apply the preprocessing function
result = preprocess_and_classify(sentence)

# Print the results
print("English words:", result['english'])
print("Other words:", result['other'])

English words: []
Other words: ['look', 'ganjar', 'kecimpung', 'lgislatif', 'eksekutif', 'bhkan', 'capres', 'mamatgs', 'berani', 'mngmcam', 'ancam', 'nyata', 'ganjar', 'nyala', 'tanda', 'bahaya', 'opm', 'operasi', 'senjata', 'mamat']


# **Saving and load Data**

In [ ]:
# # # Save the DataFrame to a CSV file
# train.to_csv(mainPath + "/Processing-Data-clean-text-4.csv", index=False)

train = pd.read_csv(cleanDataPath + "/Processing-Data-clean-text-4.csv")

# **Preprocessing with Stopword and Corpus Lexicon Tambahan**

## Text Processing **_[clean_text_5]_**

1. Menggunakan/menyempurnakan text **"Clean_text_3"** namun terdapat **tambahan processing** yaitu dengan menambhkan **stopword dan lexicon corpus** from **TimTam** yang dibuat **secara manual check by each sentences**

### Clean Lexicon Corpus

In [ ]:
# drop nilai nan di dalam corpus lexicon, yang kata formalnya tidak ada
lexicon_corpus = lexicon_corpus.dropna(subset=['formal'])

def replace_slang(text, slang_formal_pairs):
    for slang, formal in slang_formal_pairs:
        text = re.sub(r'\b{}\b'.format(re.escape(slang)), formal, text)
    return text

slang_formal_pairs = list(zip(lexicon_corpus['slang'], lexicon_corpus['formal']))

In [ ]:
# Fungsi untuk membersihkan teks
def clean_text(text, slang_formal_pairs = slang_formal_pairs):
    # Lowercasing
    text = text.lower()
    # Mengganti kata-kata dengan kata yang sesuai dari lexicon_corpus
    text = replace_slang(text, slang_formal_pairs)

    return text

In [ ]:
# Menerapkan fungsi ke kolom text
train['clean_text_5'] = train['clean_text_3'].apply(clean_text)

### Clean Stopword

In [ ]:
# # Save the DataFrame to a CSV file
# train.to_csv(cleanDataPath + "/Processing-Data-clean-text-5.csv", index=False)

train = pd.read_csv(cleanDataPath + "/Processing-Data-clean-text-5.csv")

In [ ]:
# Download stopwords and wordnet
nltk.download(['stopwords', 'wordnet'])

# Initialize Sastrawi stemmer
factory = StemmerFactory()
stemmer_sastrawi = factory.create_stemmer()

# Define stop words
stop_words = set(stopwords.words('indonesian'))
stop_words.update(stopword_corpus["Stopword"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# for i in train['clean_text_5']:
#   print(i)

In [ ]:
# Define the preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Tokenize the text into words
    words = text.split()

    # Remove stopwords and apply stemming
    processed_words = []
    for word in words:
        if word.lower() not in stop_words:  # NLTK Stopword
            stemmed_word = stemmer_sastrawi.stem(word)  # Sastrawi Stemming
            processed_words.append(stemmed_word)

    # Join the processed words back into a single string
    return " ".join(processed_words)

# Apply the preprocessing function to the 'clean_text_5' column
train['clean_text_5'] = train['clean_text_5'].astype(str).apply(preprocess_text)

# # Apply the preprocessing function to the 'clean_text_5' column
# train['clean_text_5'] = train['clean_text_5'].apply(preprocess_text)

In [ ]:
for i in train["clean_text_5"]:
  print(i)

kunjung prabowo resmi serah proyek bantu air bersih titik prabowo subianto
anies tepuk tangan riah rektor wajib mata kuliah anti korupsi putus mata rantai korupsi
dukung goblok dukung ridwan kamil skema mayoritas didik didik rendah pilih
anies sikap kritis kerja prabowo anggap sopan anggap hormat orang tua gilir gibran tengil sok kritis anggap kritis keras standar ganda
anies baswedan harap aparatur sipil negara tentara nasional indonesia polisi republik indonesia pegang sumpah milu
duh lurah dengar acara hajat rakyat puluh ribu warga kendal seru ganjar presiden hadir magnet puluh ribu warga hadir hajat rakyat ganjar mahfud antusiasme warga bukti jawa kandang banteng
prabowo menteri tahan tim tugas air universitas tahan kaji bantu air sukabumi dekade prabowo gibran
anies sibuk orang zalim jahat orang zalim
abah bapa bapa perhati didik anak anak papua pintar pintar indonesia ajar lupa warga papua bapa anies presiden
bawaslu maki jancuk warga akibat copot spanduk ganjar mahfud spanduk ca

In [ ]:
for i in train["clean_text_3"].head(10):
  print(i)

kunjung prabowo resmi serah proyek bantu air bersih titik prabowo subianto
anies tepuk tangan riah rektor wajib mata kuliah antikorupsi putus mata rantai korupsi
dukung goblok dukung ridwan kamil skema balik mayoritas didik tengah didik rendah pilih
anies sikap kritis kerja prabowo anggap sopan anggap hormat orang tua gilir gibran tengil sok kritis anggap kritis keras standar ganda
anies baswedan harap asn tni polri pegang sumpah milu
duh lurah dengar acara hajat rakyat puluh ribu warga kendal seru ganjar presiden hadir magnet puluh ribu warga hadir hajat rakyat ganjarmahfud besar antusiasme warga bukti jawa kandang banteng
prabowo kemenhan tim satgas air unhan kaji bantu air sukabumi dekade08 prabowo gibran
anies allah sibuk orang zalim jahat orang zalim
abah bapa bapa perhati didik anak anak papua pintar pintar gera indonesia ajar lupa warga papua bapa anies presiden
bawaslu maki jancuk warga gara copot spanduk ganjar mahfud spanduk capres copot bawaslu tim sukses


In [ ]:
# # Save the DataFrame to a CSV file
# train.to_csv(cleanDataPath + "/Processing-Data-clean-text-5.csv", index=False)

# train = pd.read_csv(cleanDataPath + "/Processing-Data-clean-text-5.csv")

# Processing with Another Scenario Last Time

### Text Processing **_[clean_text_6]_**

Try to Remove name in corpus with stopword, and tambahkan another lexicon based on data test submission, based on clean_text_5

In [26]:
train = pd.read_csv(cleanDataPath + "/Processing-Data-clean-text-5.csv")

In [37]:
train.isnull().sum()

text            0
label           0
clean_text_1    0
clean_text_2    0
clean_text_3    0
clean_text_4    7
clean_text_5    9
dtype: int64

In [38]:
train = train.dropna()

In [27]:
stopword_corpus = stopword_corpus.dropna().drop_duplicates().reset_index()

In [28]:
lexicon_corpus = lexicon_corpus.dropna(subset=['formal']).drop_duplicates().reset_index()

## Remove Lexicon

In [29]:
# drop nilai nan di dalam corpus lexicon, yang kata formalnya tidak ada
lexicon_corpus = lexicon_corpus.dropna(subset=['formal'])

def replace_slang(text, slang_formal_pairs):
    for slang, formal in slang_formal_pairs:
        text = re.sub(r'\b{}\b'.format(re.escape(slang)), formal, text)
    return text

slang_formal_pairs = list(zip(lexicon_corpus['slang'], lexicon_corpus['formal']))

In [30]:
# Fungsi untuk membersihkan teks
def clean_text(text, slang_formal_pairs = slang_formal_pairs):
    # Lowercasing
    text = text.lower()
    # Mengganti kata-kata dengan kata yang sesuai dari lexicon_corpus
    text = replace_slang(text, slang_formal_pairs)

    return text

In [39]:
# Menerapkan fungsi ke kolom text
train['clean_text_6'] = train['clean_text_5'].apply(clean_text)

## Clean Stopword

In [41]:
# Download stopwords and wordnet
nltk.download(['stopwords', 'wordnet'])

# Initialize Sastrawi stemmer
factory = StemmerFactory()
stemmer_sastrawi = factory.create_stemmer()

# Define stop words
stop_words = set(stopwords.words('indonesian'))
stop_words.update(stopword_corpus["Stopword"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [42]:
# Define the preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Tokenize the text into words
    words = text.split()

    # Remove stopwords and apply stemming
    processed_words = []
    for word in words:
        if word.lower() not in stop_words:  # NLTK Stopword
            stemmed_word = stemmer_sastrawi.stem(word)  # Sastrawi Stemming
            processed_words.append(stemmed_word)

    # Join the processed words back into a single string
    return " ".join(processed_words)

In [43]:
# Apply the preprocessing function to the 'clean_text_5' column
train['clean_text_6'] = train['clean_text_6'].astype(str).apply(preprocess_text)

In [45]:
# train.to_csv(cleanDataPath + "/Processing-Data-clean-text-6.csv", index=False)

## Wordcloud Visualization

In [ ]:
# Memisahkan teks berdasarkan label
texts_by_label = train.groupby('label')['clean_text_6'].apply(lambda x: ' '.join(x)).to_dict()

# Custom tokenizer to split words
def simple_tokenizer(text):
    return text.split()

In [51]:
# Vectorizer setup
vectorizer = CountVectorizer(tokenizer=simple_tokenizer, stop_words= None)

wordclouds = {}

# Membuat wordcloud untuk setiap label
for label, texts in texts_by_label.items():
    X = vectorizer.fit_transform([texts])
    word_freq = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))
    wordclouds[label] = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# Plotting the word clouds
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
axes = axes.flatten()

for ax, (label, wordcloud) in zip(axes, wordclouds.items()):
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(label, fontsize=20)
    ax.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [53]:
# Stopwords Tambahan Setelah melihat wordcloud
stopwords = ["ridwan", "kamil", "indonesia", "presiden", "pranowo", "pilih", "dukung","calon"]

# Fungsi untuk menghapus stopwords
def remove_stopwords(text, stopwords):
    return ' '.join([word for word in text.split() if word not in stopwords])

# Terapkan fungsi pada kolom clean_text_6
train['clean_text_6'] = train['clean_text_6'].apply(lambda x: remove_stopwords(x, stopwords))

In [54]:
# Memisahkan teks berdasarkan label
texts_by_label = train.groupby('label')['clean_text_6'].apply(lambda x: ' '.join(x)).to_dict()

# Custom tokenizer to split words
def simple_tokenizer(text):
    return text.split()

In [55]:
# Vectorizer setup
vectorizer = CountVectorizer(tokenizer=simple_tokenizer, stop_words= None)

wordclouds = {}

# Membuat wordcloud untuk setiap label
for label, texts in texts_by_label.items():
    X = vectorizer.fit_transform([texts])
    word_freq = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))
    wordclouds[label] = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# Plotting the word clouds
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
axes = axes.flatten()

for ax, (label, wordcloud) in zip(axes, wordclouds.items()):
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(label, fontsize=20)
    ax.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [56]:
# train.to_csv(cleanDataPath + "/Processing-Data-clean-text-6.csv", index=False)

In [59]:
print("Hello World")

Hello World
